In [2]:
import pandas as pd

from Bio import SeqIO

import os
from Bio.Blast.Applications import NcbiblastpCommandline

import seaborn as sns

In [3]:
sns.violinplot

<function seaborn.categorical.violinplot(*, x=None, y=None, hue=None, data=None, order=None, hue_order=None, bw='scott', cut=2, scale='area', scale_hue=True, gridsize=100, width=0.8, inner='box', split=False, dodge=True, orient=None, linewidth=None, color=None, palette=None, saturation=0.75, ax=None, **kwargs)>